# Homework 2

In [83]:
import numpy as np
import pandas as pd

import requests
from bs4 import BeautifulSoup

## Paramethers for requests:
    1- ww_x_UNITE_ACAD - Section (Informatique?)  =  249847
    2- ww_x_PERIODE_ACAD - Semester wanted        =  978181
    3- ww_i_reportModel - General parameter =  133685247

In [84]:
unite_academique=249847
periode_academique=[978181, 978187,978195,39486325,123455150,213637754,213637922,213638028,355925344]
report_model=133685247



In [85]:
base_link="http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_b_list=1&ww_i_reportmodel=133685247&ww_c_langue=&ww_i_reportModelXsl=133685270&zz_x_UNITE_ACAD=Informatique&ww_x_UNITE_ACAD=249847&zz_x_PERIODE_ACAD=2007-2008&ww_x_PERIODE_ACAD=978181&zz_x_PERIODE_PEDAGO=&ww_x_PERIODE_PEDAGO=null&zz_x_HIVERETE=&ww_x_HIVERETE=null&dummy=ok"
base='http://isa.epfl.ch/imoniteur_ISAP/%21gedpublicreports.htm?ww_i_reportmodel=133685247'
base2='http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_b_list=1&ww_i_reportmodel=133685247&ww_c_langue=&ww_i_reportModelXsl=133685270&zz_x_UNITE_ACAD=Informatique'


In [86]:
payload = {'ww_x_UNITE_ACAD': '249847'}
r = requests.get(base2,params=payload)
print(r)

<Response [200]>


In [87]:
print(r.url)

http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_b_list=1&ww_i_reportmodel=133685247&ww_c_langue=&ww_i_reportModelXsl=133685270&zz_x_UNITE_ACAD=Informatique&ww_x_UNITE_ACAD=249847


In [88]:
r.text

'<html><head><META http-equiv="Content-Type" content="text/html; charset=iso-8859-1"><div></div><title></title><script type="text/javascript" src="GEDPUBLICREPORTS.txt?ww_x_path=Gestac.Base.Palette_js&amp;ww_c_langue=fr"></script><link rel="stylesheet" type="text/css" href="GEDPUBLICREPORTS.css?ww_x_path=Gestac.Moniteur.Style"><link rel="stylesheet" type="text/css" href="GEDPUBLICREPORTS.css?ww_x_path=Gestac.Moniteur.StyleNavigator"></head><body bgcolor="#ffffff" marginheight="0" marginwidth="5" link="#666666" vlink="#666666" alink="#666666"><div class="filtres"><form method="GET" name="f" action="!GEDPUBLICREPORTS.filter"><input type="hidden" name="ww_b_list" value="1"><input type="hidden" name="ww_i_reportmodel" value="133685247"><input type="hidden" name="ww_c_langue" value=""><h1 id="titre">Liste des étudiants inscrits par semestre</h1><table border="0" id="format"><tr><th>Format:</th></tr><tr><td><input type="radio" name="ww_i_reportModelXsl" value="133685270" checked>html</td></t

In [89]:
a=r.url
print(type(a))

<class 'str'>


In [90]:
b='http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_b_list=1&ww_i_reportmodel=133685247&ww_c_langue=&ww_i_reportModelXsl=133685270&zz_x_UNITE_ACAD=Informatique&ww_x_UNITE_ACAD=249847'


In [91]:
print(a==b)

True


In [92]:
semester_bachelor=['Bachelor semestre 1','Bachelor semestre 2','Bachelor semestre 3','Bachelor semestre 4','Bachelor semestre 5','Bachelor semestre 6']
semester_master=['Master semestre 1','Master semestre 2','Master semestre 3']
years=['2007-2008','2008-2009','2009-2010','2010-2011','2011-2012','2012-2013','2013-2014','2014-2015','2015-2016','2016-2017']

In [169]:
# Generate dictionary
gps_dictionary={}
for year in years:
    for semester in semester_bachelor:
        gps_dictionary['Informatique, '+year+', '+semester]=0
    for semester in semester_master:
        gps_dictionary['Informatique, '+year+', '+semester]=0

In [95]:
soup=BeautifulSoup(r.text, "lxml")

In [144]:
tag=soup.body

gps_list=tag.find_all('a')

{'onclick': "loadReport('ww_x_GPS=-1');return false;", 'href': 'javascript:void(0)', 'class': ['ww_x_GPS']}
{'onclick': "loadReport('ww_x_GPS=24065660');return false;", 'href': 'javascript:void(0)', 'class': ['ww_x_GPS']}
{'onclick': "loadReport('ww_x_GPS=24065672');return false;", 'href': 'javascript:void(0)', 'class': ['ww_x_GPS']}
{'onclick': "loadReport('ww_x_GPS=39467937');return false;", 'href': 'javascript:void(0)', 'class': ['ww_x_GPS']}
{'onclick': "loadReport('ww_x_GPS=39467957');return false;", 'href': 'javascript:void(0)', 'class': ['ww_x_GPS']}
{'onclick': "loadReport('ww_x_GPS=71454914');return false;", 'href': 'javascript:void(0)', 'class': ['ww_x_GPS']}
{'onclick': "loadReport('ww_x_GPS=71454935');return false;", 'href': 'javascript:void(0)', 'class': ['ww_x_GPS']}
{'onclick': "loadReport('ww_x_GPS=117663512');return false;", 'href': 'javascript:void(0)', 'class': ['ww_x_GPS']}
{'onclick': "loadReport('ww_x_GPS=117663545');return false;", 'href': 'javascript:void(0)', '

In [167]:
# Process the GPS list
# Remove the ones without "Informatique", "Master" or "Bachelor"
# Hard preprocessing to fast eliminate the unwanted strings
gps_processed=[]
key1='Informatique'
key2='Bachelor'
key3='Master'
for i in gps_list:
    if key1 in i.contents[0] and (key2 in i.contents[0] or key3 in i.contents[0]):
        gps_processed.append(i)


In [171]:
# Keep only the right ones - get the GPS in the gps_dictionary
for key in gps_dictionary:
    for item in gps_processed:
        if key==item.contents[0]:
            gps=str(item).split("'")[1].split("=")[1]
            gps_dictionary[key]=gps


In [ ]:
parameters={'ww_i_reportModel'='133685247','ww_i_reportModelXsl'='133685270','ww_x_UNITE_ACAD','249847'}